In [23]:
import re
import numpy as np
import os
from tqdm import tqdm

os.chdir("/Users/noracai/Documents/_coding_projects/IBM-Models/jane-eyre")
print(os.getcwd()) 


/Users/noracai/Documents/_coding_projects/IBM-Models/jane-eyre


## Preprocess

### Gale_church_alignment

In [ ]:
# def gale_church_align(english_sentences, french_sentences, merge_threshold=5):
#     """
#     Aligns English and French sentences using an improved Gale-Church approach.
#     - `merge_threshold`: Merges short sentences below this word count.
#     - Ensures no blank lines and merges small misaligned fragments.
#     """
#     eng_len = len(english_sentences)
#     fr_len = len(french_sentences)

#     # Convert sentences to word count lengths
#     eng_lengths = np.array([len(sent.split()) for sent in english_sentences])
#     fr_lengths = np.array([len(sent.split()) for sent in french_sentences])

#     # Create a cost matrix based on word count differences
#     cost_matrix = np.abs(eng_lengths[:, None] - fr_lengths[None, :])

#     # Initialize DP table (INF for all cells except [0,0])
#     dp = np.full((eng_len + 1, fr_len + 1), float('inf'))
#     dp[0, 0] = 0

#     # Back-pointer matrix for path reconstruction
#     backtrace = np.zeros((eng_len + 1, fr_len + 1), dtype=int)

#     # Fill DP table using minimal cost merging strategy
#     for i in range(eng_len + 1):
#         for j in range(fr_len + 1):
#             if i > 0 and j > 0:  # Normal 1:1 alignment
#                 cost = cost_matrix[i - 1, j - 1]
#                 if dp[i - 1, j - 1] + cost < dp[i, j]:
#                     dp[i, j] = dp[i - 1, j - 1] + cost
#                     backtrace[i, j] = 1  # (1 English -> 1 French)

#             if i > 0:  # Merge English sentence
#                 cost = cost_matrix[i - 1, max(0, j - 1)] if j > 0 else 0
#                 if dp[i - 1, j] + cost < dp[i, j]:
#                     dp[i, j] = dp[i - 1, j] + cost
#                     backtrace[i, j] = 2  # (1 English -> Multiple French)

#             if j > 0:  # Merge French sentence
#                 cost = cost_matrix[max(0, i - 1), j - 1] if i > 0 else 0
#                 if dp[i, j - 1] + cost < dp[i, j]:
#                     dp[i, j] = dp[i, j - 1] + cost
#                     backtrace[i, j] = 3  # (Multiple English -> 1 French)

#     # Backtrace to reconstruct aligned sentence pairs
#     aligned_sentences = []
#     i, j = eng_len, fr_len
#     eng_buffer, fr_buffer = [], []

#     while i > 0 or j > 0:
#         if backtrace[i, j] == 1:  # Normal 1:1 alignment
#             eng_buffer.append(english_sentences[i - 1])
#             fr_buffer.append(french_sentences[j - 1])
#             aligned_sentences.append((" ".join(reversed(eng_buffer)), " ".join(reversed(fr_buffer))))
#             eng_buffer, fr_buffer = [], []  # Reset buffers
#             i -= 1
#             j -= 1

#         elif backtrace[i, j] == 2:  # Merge English sentence
#             eng_buffer.append(english_sentences[i - 1])
#             i -= 1

#         elif backtrace[i, j] == 3:  # Merge French sentence
#             fr_buffer.append(french_sentences[j - 1])
#             j -= 1

#     # Reverse since we constructed it backwards
#     aligned_sentences.reverse()

#     # **Remove empty sentence pairs to prevent blank lines**
#     aligned_sentences = [(e, f) for e, f in aligned_sentences if e.strip() or f.strip()]

#     return aligned_sentences

In [20]:
def clean_text(text):
    """Enhanced cleaning function for dialogue handling"""
    # Remove quotation marks
    text = re.sub(r'[«»""“”]', '', text)
    
    # Standardize dialogue formatting - replace dialogue dashes with a marker
    text = re.sub(r'\s*-\s+', ' ', text)  # Remove " - " pattern
    text = re.sub(r'^\s*-\s+', '', text)  # Remove dash at beginning of text
    text = re.sub(r'\n\s*-\s+', '\n', text)  # Remove dash at beginning of lines
    text = re.sub(r'\s*—\s+', ' ', text)  # Remove " - " pattern
    text = re.sub(r'^\s*—\s+', '', text)  # Remove dash at beginning of text
    text = re.sub(r'\n\s*—\s+', '\n', text)  # Remove dash at beginning of lines
    text = re.sub(r'\s*–\s+', ' ', text)  # Remove " - " pattern
    text = re.sub(r'^\s*–\s+', '', text)  # Remove dash at beginning of text
    text = re.sub(r'\n\s*–\s+', '\n', text)  # Remove dash at beginning of lines
    text = re.sub(r'_(\w+)_', r'\1', text)

    text = re.sub(r'\s+([?!;:])', r'\1', text)
    # Remove parenthetical comments which often appear in one language but not the other
    text = re.sub(r'\([^)]*\)', '', text)
    
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
def split_into_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    abbreviations = {"Mr.", "Mrs.", "Ms.", "Dr.", "Jr.", "Sr.", "St.", "Prof.", "Capt.", "Lt.", "Col.", "Gen.", "Sgt.", "Mt.", "M.", "Sr."}

    merged_sentences = []
    buffer = ""

    for sentence in sentences:
        # Handle abbreviation cases
        if buffer:
            sentence = buffer + " " + sentence
            buffer = ""

        if any(sentence.endswith(abbr) for abbr in abbreviations):
            buffer = sentence  # Hold onto this part to merge with the next
            continue

        # Remove em dash if it's at the start of a sentence
        sentence = re.sub(r'\s*-\s+', ' ', sentence)  # Remove " - " pattern
        sentence = re.sub(r'^\s*-\s+', '', sentence)  # Remove dash at beginning of text
        sentence = re.sub(r'\n\s*-\s+', '\n', sentence)  # Remove dash at beginning of lines
        sentence = re.sub(r'^\s*—\s+', '', sentence)  # Remove dash at beginning of text
        sentence = re.sub(r'\n\s*—\s+', '\n', sentence)  # Remove dash at beginning of lines
        sentence = re.sub(r'\s+', ' ', sentence).strip()

        if sentence.startswith(","):
            if merged_sentences:
                merged_sentences[-1] += " " + sentence  # Merge with the previous sentence
            else:
                merged_sentences.append(sentence)  # Edge case: if it's the first sentence
        else:
            merged_sentences.append(sentence)

    if buffer:
        merged_sentences.append(buffer)  # Add any leftover buffer

    return merged_sentences

In [57]:
import numpy as np

def gale_church_align(english_sentences, french_sentences, skip_weight=2.0):
    """
    Aligns English and French sentences using a dynamic programming approach.
    
    Allowed moves:
      (1,1): Align one English sentence with one French sentence.
      (1,0): Skip a French sentence (i.e. align an English sentence alone).
      (0,1): Skip an English sentence (i.e. align a French sentence alone).
      
    The cost for a 1:1 alignment is the absolute difference in word counts.
    The skip cost is the sentence's word count multiplied by skip_weight.
    
    A heavy penalty is applied on the very first skip move to force the first 
    English and French sentences to be aligned.
    
    Returns a list of aligned blocks (pairs). Note that these blocks may contain
    multiple sentences from each side (i.e. merged blocks), but increasing skip_weight
    will tend to force a more granular (less merged) alignment.
    """
    eng_len = len(english_sentences)
    fr_len = len(french_sentences)
    
    # Compute sentence lengths (word counts)
    eng_lengths = np.array([len(sent.split()) for sent in english_sentences])
    fr_lengths = np.array([len(sent.split()) for sent in french_sentences])
    
    # Build a cost matrix based on absolute differences in word counts
    cost_matrix = np.abs(eng_lengths[:, None] - fr_lengths[None, :])
    
    # Initialize DP and backtrace tables.
    dp = np.full((eng_len+1, fr_len+1), float('inf'))
    backtrace = np.zeros((eng_len+1, fr_len+1), dtype=int)
    dp[0,0] = 0
    
    # --- Boundary Initialization with heavy penalty on first skip ---
    for i in range(1, eng_len+1):
        if i == 1:
            dp[i, 0] = dp[i-1, 0] + 1000 * eng_lengths[i-1]  # heavy penalty
        else:
            dp[i, 0] = dp[i-1, 0] + skip_weight * eng_lengths[i-1]
        backtrace[i,0] = 2  # (1,0) move
    for j in range(1, fr_len+1):
        if j == 1:
            dp[0, j] = dp[0, j-1] + 1000 * fr_lengths[j-1]  # heavy penalty
        else:
            dp[0, j] = dp[0, j-1] + skip_weight * fr_lengths[j-1]
        backtrace[0,j] = 3  # (0,1) move
    # --- End Boundary Initialization ---
    
    # Fill DP table for i>=1 and j>=1.
    for i in range(1, eng_len+1):
        for j in range(1, fr_len+1):
            # Option 1: (1,1) alignment.
            cost_11 = cost_matrix[i-1, j-1]
            if dp[i-1, j-1] + cost_11 < dp[i,j]:
                dp[i,j] = dp[i-1,j-1] + cost_11
                backtrace[i,j] = 1
            # Option 2: (1,0) skip French.
            cost_10 = skip_weight * eng_lengths[i-1]
            if dp[i-1, j] + cost_10 < dp[i,j]:
                dp[i,j] = dp[i-1, j] + cost_10
                backtrace[i,j] = 2
            # Option 3: (0,1) skip English.
            cost_01 = skip_weight * fr_lengths[j-1]
            if dp[i, j-1] + cost_01 < dp[i,j]:
                dp[i,j] = dp[i, j-1] + cost_01
                backtrace[i,j] = 3
    
    # Backtrace to reconstruct the alignment.
    aligned_blocks = []
    i, j = eng_len, fr_len
    eng_buffer, fr_buffer = [], []
    
    while i > 0 or j > 0:
        if i > 0 and j > 0 and backtrace[i,j] == 1:
            # (1,1) move: flush buffers.
            eng_buffer.append(english_sentences[i-1])
            fr_buffer.append(french_sentences[j-1])
            aligned_blocks.append((" ".join(reversed(eng_buffer)),
                                     " ".join(reversed(fr_buffer))))
            eng_buffer, fr_buffer = [], []
            i -= 1
            j -= 1
        elif i > 0 and (j == 0 or backtrace[i,j] == 2):
            # (1,0): Skip French.
            eng_buffer.append(english_sentences[i-1])
            i -= 1
        elif j > 0 and (i == 0 or backtrace[i,j] == 3):
            # (0,1): Skip English.
            fr_buffer.append(french_sentences[j-1])
            j -= 1
        else:
            break
    
    # If any leftover sentences remain, add them as one block.
    if eng_buffer or fr_buffer:
        aligned_blocks.insert(0, (" ".join(reversed(eng_buffer)),
                                    " ".join(reversed(fr_buffer))))
    
    # Reverse the blocks to restore original order.
    aligned_blocks.reverse()
    # Remove any completely empty blocks.
    aligned_blocks = [(e, f) for e, f in aligned_blocks if e.strip() or f.strip()]
    return aligned_blocks

In [59]:
# working version 

def gale_church_align(english_sentences, french_sentences, merge_threshold=5):
    """
    Aligns English and French sentences using a dynamic programming approach.
    
    This version uses three moves:
      (1,1): Align one English sentence with one French sentence.
      (1,0): Skip a French sentence (i.e. align an English sentence alone).
      (0,1): Skip an English sentence (i.e. align a French sentence alone).
      
    A heavy penalty is applied on the very first skip move to force the first
    sentences on both sides to align.
    
    Returns a list of aligned pairs (blocks). Each block may contain multiple 
    sentences from each language; the sum of sentences across blocks equals the 
    original counts.
    """
    eng_len = len(english_sentences)
    fr_len = len(french_sentences)

    # Compute sentence lengths (using word counts)
    eng_lengths = np.array([len(sent.split()) for sent in english_sentences])
    fr_lengths = np.array([len(sent.split()) for sent in french_sentences])
    
    # Create a cost matrix based on absolute differences in word counts
    cost_matrix = np.abs(eng_lengths[:, None] - fr_lengths[None, :])
    
    # Initialize DP table and backtrace table
    dp = np.full((eng_len + 1, fr_len + 1), float('inf'))
    backtrace = np.zeros((eng_len + 1, fr_len + 1), dtype=int)
    dp[0, 0] = 0

    # --- Boundary Initialization with heavy penalty on first skip ---
    for i in range(1, eng_len + 1):
        if i == 1:
            # Heavy penalty for skipping the first English sentence
            dp[i, 0] = dp[i - 1, 0] + 1000 * eng_lengths[i - 1]
        else:
            dp[i, 0] = dp[i - 1, 0] + eng_lengths[i - 1]
        backtrace[i, 0] = 2  # Move: (1,0)
    for j in range(1, fr_len + 1):
        if j == 1:
            # Heavy penalty for skipping the first French sentence
            dp[0, j] = dp[0, j - 1] + 1000 * fr_lengths[j - 1]
        else:
            dp[0, j] = dp[0, j - 1] + fr_lengths[j - 1]
        backtrace[0, j] = 3  # Move: (0,1)
    # --- End Boundary Initialization ---

    # Fill DP table for cells (i>=1, j>=1)
    for i in range(1, eng_len + 1):
        for j in range(1, fr_len + 1):
            # Option 1: Align one English sentence with one French sentence (1,1)
            cost = cost_matrix[i - 1, j - 1]
            if dp[i - 1, j - 1] + cost < dp[i, j]:
                dp[i, j] = dp[i - 1, j - 1] + cost
                backtrace[i, j] = 1  # (1,1)
            # Option 2: Skip French sentence (merge English sentence)
            cost = cost_matrix[i - 1, j - 1]
            if dp[i - 1, j] + cost < dp[i, j]:
                dp[i, j] = dp[i - 1, j] + cost
                backtrace[i, j] = 2  # (1,0)
            # Option 3: Skip English sentence (merge French sentence)
            cost = cost_matrix[i - 1, j - 1]
            if dp[i, j - 1] + cost < dp[i, j]:
                dp[i, j] = dp[i, j - 1] + cost
                backtrace[i, j] = 3  # (0,1)

    # Backtrace to reconstruct aligned blocks
    aligned_sentences = []
    i, j = eng_len, fr_len
    eng_buffer, fr_buffer = [], []
    
    while i > 0 or j > 0:
        if i > 0 and j > 0 and backtrace[i, j] == 1:
            # (1,1): Align both; flush buffers into a block
            eng_buffer.append(english_sentences[i - 1])
            fr_buffer.append(french_sentences[j - 1])
            aligned_sentences.append((" ".join(reversed(eng_buffer)),
                                      " ".join(reversed(fr_buffer))))
            eng_buffer, fr_buffer = [], []
            i -= 1
            j -= 1
        elif i > 0 and (j == 0 or backtrace[i, j] == 2):
            # (1,0): Skip French, accumulate English sentence.
            eng_buffer.append(english_sentences[i - 1])
            i -= 1
        elif j > 0 and (i == 0 or backtrace[i, j] == 3):
            # (0,1): Skip English, accumulate French sentence
            fr_buffer.append(french_sentences[j - 1])
            j -= 1
        else:
            break

    # If any leftover sentences remain, add them as one block
    if eng_buffer or fr_buffer:
        aligned_sentences.insert(0, (" ".join(reversed(eng_buffer)),
                                       " ".join(reversed(fr_buffer))))

    # Reverse the aligned blocks to restore original order
    aligned_sentences.reverse()
    
    # Remove any completely empty blocks
    aligned_sentences = [(e, f) for e, f in aligned_sentences if e.strip() or f.strip()]
    return aligned_sentences

In [60]:
def process_and_align(english_input, french_input, english_output, french_output, merge_threshold=50):
    
    with open(english_input, 'r', encoding='utf-8') as eng_in, \
         open(french_input, 'r', encoding='utf-8') as fr_in:
        
        english_text = ' '.join([line.strip() for line in eng_in if line.strip()])
        french_text = ' '.join([line.strip() for line in fr_in if line.strip()]) 

        english_text = clean_text(english_text)
        french_text = clean_text(french_text)

        english_sentences = split_into_sentences(english_text)
        french_sentences = split_into_sentences(french_text)

    print(f"Initial sentence count: English ({len(english_sentences)}), French ({len(french_sentences)})")
    aligned_sentences = gale_church_align(english_sentences, french_sentences)

    num_aligned = len(aligned_sentences)
    print(f"Aligned sentence count: {num_aligned}")
    
    # Check if there's a mismatch
    eng_count = sum(1 for eng, _ in aligned_sentences if eng.strip())
    fr_count = sum(1 for _, fr in aligned_sentences if fr.strip())

    if eng_count != fr_count:
        print(f"Sentence mismatch after alignment! EN = {eng_count}, FR = {fr_count}")
        
    with open(english_output, 'w', encoding='utf-8') as eng_out, \
         open(french_output, 'w', encoding='utf-8') as fr_out:
        
        for eng_sentence, fr_sentence in aligned_sentences:
            eng_out.write(eng_sentence.strip() + '\n')
            fr_out.write(fr_sentence.strip() + '\n')

    print(f"✅ Complete: English file: {english_output}, French file: {french_output}")

### Process and align

### french

In [56]:
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/Fr_1854_Lesbazeilles_Souvestre.txt', 'Fr_1854_Lesbazeilles_Souvestre.e','Fr_1854_Lesbazeilles_Souvestre.f')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/Fr_1919_Gilbert_Duvivier.txt', 'Fr_1919_Gilbert_Duvivier.e','Fr_1919_Gilbert_Duvivier.f')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/Fr_1946_Redon_Dulong.txt', 'Fr_1946_Redon_Dulong.e','Fr_1946_Redon_Dulong.f')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/Fr_1964_Maurat.txt', 'Fr_1964_Maurat.e','Fr_1964_Maurat.f')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/Fr_1966_Monod.txt', 'Fr_1966_Monod.e','Fr_1966_Monod.f')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/Fr_2008_Jean.txt', 'Fr_2008_Jean.e','Fr_2008_Jean.f')

Initial sentence count: English (9621), French (9493)
Aligned sentence count: 7367
✅ Complete: English file: Fr_1854_Lesbazeilles_Souvestre.e, French file: Fr_1854_Lesbazeilles_Souvestre.f
Initial sentence count: English (9621), French (25680)
Aligned sentence count: 8364
✅ Complete: English file: Fr_1919_Gilbert_Duvivier.e, French file: Fr_1919_Gilbert_Duvivier.f
Initial sentence count: English (9621), French (9829)
Aligned sentence count: 7458
✅ Complete: English file: Fr_1946_Redon_Dulong.e, French file: Fr_1946_Redon_Dulong.f
Initial sentence count: English (9621), French (10469)
Aligned sentence count: 8715
✅ Complete: English file: Fr_1964_Maurat.e, French file: Fr_1964_Maurat.f
Initial sentence count: English (9621), French (9940)
Aligned sentence count: 9180
✅ Complete: English file: Fr_1966_Monod.e, French file: Fr_1966_Monod.f
Initial sentence count: English (9621), French (13488)
Aligned sentence count: 8600
✅ Complete: English file: Fr_2008_Jean.e, French file: Fr_2008_Jean

### Italian

In [61]:
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_1904_Anon.txt', 'Italian/It_1904_Anon.e','Italian/It_1904_Anon.i')

Initial sentence count: English (9621), French (9577)
Aligned sentence count: 7288
✅ Complete: English file: Italian/It_1904_Anon.e, French file: Italian/It_1904_Anon.i


In [62]:
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_1951_Pozzo_Galeazzi.txt', 'It_1951_Pozzo_Galeazzi.e','It_1951_Pozzo_Galeazzi.i')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_1956_SpaventaFilippi.txt', 'It_1956_SpaventaFilippi.e','It_1956_SpaventaFilippi.i')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_1974_Dettore.txt', 'It_1974_Dettore.e','It_1974_Dettore.i')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_2008_Lamberti.txt', 'It_2008_Lamberti.e','It_2008_Lamberti.i')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_2011_DEzio.txt', 'It_2011_DEzio.e','It_2011_DEzio.i')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_2014_Pareschi.txt', 'It_2014_Pareschi.e','It_2014_Pareschi.i')
process_and_align('full_texts/English_gutenburg.txt', 'full_texts/It_2014_Sacchini.txt', 'It_2014_Sacchini.e','It_2014_Sacchini.i')

Initial sentence count: English (9621), French (10308)
Aligned sentence count: 8385
✅ Complete: English file: It_1951_Pozzo_Galeazzi.e, French file: It_1951_Pozzo_Galeazzi.i
Initial sentence count: English (9621), French (12215)
Aligned sentence count: 7948
✅ Complete: English file: It_1956_SpaventaFilippi.e, French file: It_1956_SpaventaFilippi.i
Initial sentence count: English (9621), French (10676)
Aligned sentence count: 8732
✅ Complete: English file: It_1974_Dettore.e, French file: It_1974_Dettore.i
Initial sentence count: English (9621), French (12537)
Aligned sentence count: 8076
✅ Complete: English file: It_2008_Lamberti.e, French file: It_2008_Lamberti.i
Initial sentence count: English (9621), French (12710)
Aligned sentence count: 8438
✅ Complete: English file: It_2011_DEzio.e, French file: It_2011_DEzio.i
Initial sentence count: English (9621), French (11065)
Aligned sentence count: 8906
✅ Complete: English file: It_2014_Pareschi.e, French file: It_2014_Pareschi.i
Initial se

# Miscs

## Combine chapters

In [ ]:
# def combine_texts(folder_path, output_file):
#     files = sorted([f for f in os.listdir(folder_path) if f.endswith(".txt")])

#     with open(output_file, "w", encoding="utf-8") as outfile:
#         for i, f in enumerate(files, start=1): 
#             file_path = os.path.join(folder_path, f)
#             with open(file_path, "r", encoding="utf-8") as infile:
#                 text = (infile.read())
#                 outfile.write(f"Capitúlo {i}\n\n{text.strip()}\n\n")

#     print(f"Combined text saved to {output_file}")
#     return output_file

# combine_texts('original_texts/Spanish_1928_anon', 'full_texts/Spanish_1928_anon.txt')

In [ ]:
# # manual alignment
# from itertools import zip_longest

# # Load English and French files
# with open('aligned-ch1.e', 'r', encoding='utf-8') as eng_file:
#     english_sentences = eng_file.readlines()

# with open('aligned-ch1.f', 'r', encoding='utf-8') as fr_file:
#     french_sentences = fr_file.readlines()

# # Compare sentences line by line
# for idx, (eng_sent, fr_sent) in enumerate(zip_longest(english_sentences, french_sentences, fillvalue='')):
#     eng_sent = eng_sent.strip()
#     fr_sent = fr_sent.strip()
#     eng_len = len(eng_sent)
#     fr_len = len(fr_sent)

#     # Check for significant difference (e.g., one is 1.5x longer)
#     if fr_len > 1.5 * eng_len:
#         status = "⚠️ French significantly longer"
#     elif eng_len > 1.5 * fr_len:
#         status = "⚠️ English significantly longer"
#     elif fr_len == eng_len == 0: 
#         continue
#     else:
#         status = "✅ Aligned"

#     # Display the comparison
#     print(f"Line {idx + 1}: {status}")
#     print(f"EN: {eng_sent}")
#     print(f"FR: {fr_sent}")
#     print("-" * 50)

FileNotFoundError: [Errno 2] No such file or directory: 'aligned-ch1.e'